<a href="https://colab.research.google.com/github/arkaprabha10/Samsung-PRISM/blob/main/VAE_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install python_speech_features
!pip install --upgrade librosa

80 MEL FILTERBANK FEATURES METHOD 1


In [ ]:
import librosa
import numpy
import scipy.io.wavfile
from scipy.fftpack import dct
from scipy import signal

audio_path = '/input.wav'
sample_rate, signal = scipy.io.wavfile.read(audio_path)
y , sr = librosa.load(audio_path,sr=None)
pre_emphasis = 0.97
emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
frame_size = 0.025
frame_stride = 0.01

frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame

pad_signal_length = num_frames * frame_step + frame_length
z = numpy.zeros((pad_signal_length - signal_length))
pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal

#indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
#frames = pad_signal[indices.astype(numpy.int32, copy=False)]

#frames *= numpy.hanning(frame_length)
#print(emphasized_signal.shape)
# direct conversion doesnt use hamming window
#uncomment below to directly use librosa feature
#print(ans.shape)
print(pad_signal.shape)

(55120,)


80 MEL SCALE OPTION 2 (continued from above)



In [ ]:
print(frame_length)
for i in range (0,len(pad_signal),frame_step):
  temp=pad_signal[i:frame_length+i]
  temp*=numpy.hanning(len(temp))
  for j in range(len(temp)):
    pad_signal[i+j]=temp[j]

ans=librosa.feature.melspectrogram(y=pad_signal, sr=sr,n_mels=80)  
print(pad_signal.shape)

400
(55120,)


In [ ]:
NFFT = 512
mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum

nfilt = 80

low_freq_mel = 0
high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
for m in range(1, nfilt + 1):
    f_m_minus = int(bin[m - 1])   # left
    f_m = int(bin[m])             # center
    f_m_plus = int(bin[m + 1])    # right

    for k in range(f_m_minus, f_m):
        fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
    for k in range(f_m, f_m_plus):
        fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
filter_banks = numpy.dot(pow_frames, fbank.T)
filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
filter_banks = 20 * numpy.log10(filter_banks)  # dB

filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
filter_banks=numpy.transpose(filter_banks)
print(filter_banks.shape)

(80, 342)


In [ ]:
from scipy import signal
f_array, t_array, coeff_array = signal.stft(frames, sample_rate)
print(coeff_array.shape)
print(t_array)
print(len(t_array))

(342, 129, 5)
[0.    0.008 0.016 0.024 0.032]
5


In [ ]:
from librosa import *
import librosa.feature.inverse
S = librosa.feature.inverse.mel_to_audio(ans,sr)


In [ ]:
from scipy.io.wavfile import write
write('/test1.wav', sr, S)

In [ ]:
print(S.shape)

(54784,)
